In [1]:
import plotly.express as px
import plotly.graph_objects as go
#data imports
import pandas as pd
import numpy as np
import os
import pandas_datareader.data as web
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='YOUR_API_KEY',output_format='pandas', indexing_type='date')

In [2]:
def adjust(date, close, adj_close, in_col, rounding=4):
    '''
    If using forex or Crypto - Change the rounding accordingly!
    '''
    try:
        factor = adj_close / close
        return round(in_col * factor, rounding)
    except ZeroDivisionError:
        print('WARNING: DIRTY DATA >> {} Close: {} | Adj Close {} | in_col: {}'.format(date, close, adj_close, in_col))
        return 0

In [5]:
df, meta_data = ts.get_daily_adjusted('BABA', outputsize='full')    
df.sort_index(ascending=True, inplace = True)

df['adj close'] = df['5. adjusted close']
df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])    
#calculate daily returns
df['daily_return'] = df['adj close'].pct_change()
# calculate cumluative return
df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())

In [6]:
df

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,adj close,adj open,adj high,adj low,daily_return,cumluative_return
date,,,,,,,,,,,,,,
2014-09-19,92.70,99.700,89.95,93.89,93.89,271879400.0,0.0,1.0,93.89,92.70,99.700,89.95,NaN,NaN
2014-09-22,92.70,92.950,89.50,89.89,89.89,66332498.0,0.0,1.0,89.89,92.70,92.950,89.50,-0.042603,0.957397
2014-09-23,88.94,90.480,86.62,87.17,87.17,38861975.0,0.0,1.0,87.17,88.94,90.480,86.62,-0.030259,0.928427
2014-09-24,88.47,90.570,87.22,90.57,90.57,31864232.0,0.0,1.0,90.57,88.47,90.570,87.22,0.039004,0.964639
2014-09-25,91.09,91.500,88.50,88.92,88.92,28526179.0,0.0,1.0,88.92,91.09,91.500,88.50,-0.018218,0.947066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-25,225.00,229.240,222.50,222.72,222.72,24819815.0,0.0,1.0,222.72,225.00,229.240,222.50,-0.029923,2.372138
2021-03-26,222.00,229.239,220.42,227.26,227.26,22030750.0,0.0,1.0,227.26,222.00,229.239,220.42,0.020384,2.420492
2021-03-29,225.49,232.380,225.10,231.86,231.86,18029105.0,0.0,1.0,231.86,225.49,232.380,225.10,0.020241,2.469486


In [24]:
def getTickerData(ticker, start,end):
    df, meta_data = ts.get_daily_adjusted(ticker, outputsize='full')    
    df.sort_index(ascending=True, inplace = True)
    
    df = df[start:end]
    
    df['adj close'] = df['5. adjusted close']
    df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
    df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
    df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])    
    #calculate daily returns
    df['daily_return'] = df['adj close'].pct_change()
    # calculate cumluative return
    df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())
    
    return df[['adj open','adj close','adj high', 'adj low']]

In [23]:
getTickerData('DIS','2021-01-01','2021-04-01')

,adj open,adj close,adj high,adj low
date,,,,
2021-01-04,182.255,177.68,182.5900,174.7300
2021-01-05,176.200,178.44,180.3399,176.2000
2021-01-06,178.320,179.12,181.6100,177.2836
2021-01-07,179.500,178.58,179.9500,177.5750
2021-01-08,178.810,178.69,180.0200,176.6000
...,...,...,...,...
2021-03-25,181.850,186.91,187.7100,181.0100
2021-03-26,188.830,185.92,188.9750,182.5700
2021-03-29,185.240,184.85,185.9400,183.1500
